In [52]:
import numpy as np
import math

import keras
from keras.models import  Sequential
from keras.layers import Activation, BatchNormalization
from keras.layers.core import Dense
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy

from rdkit import Chem, DataStructs
from rdkit.Chem import rdMolDescriptors, Draw
from rdkit.Chem.Draw import IPythonConsole
from sklearn.preprocessing import MinMaxScaler

In [53]:
'''DATA PREPARE'''
#read data file
with open('1k_SMILES_strings.txt') as f:
    array_SMILES_train = f.readlines()
with open('1k_SMILES_strings_charges.txt') as f:
    array_charges_train = f.readlines()
with open('10_SMILES_strings_test.txt') as f:
    array_SMILES_test = f.readlines()

In [54]:
#convert SMILES strings into molecules
mols_train = [Chem.rdmolfiles.MolFromSmiles(SMILES_string_train) for SMILES_string_train in array_SMILES_train]
mols_test = [Chem.rdmolfiles.MolFromSmiles(SMILES_string_test) for SMILES_string_test in array_SMILES_test]

In [55]:
#convert molecules into fingerprints
bi = {}
fps_train = [rdMolDescriptors.GetMorganFingerprintAsBitVect(m_train,radius=2, bitInfo=bi, nBits=256) for m_train in mols_train]
fps_test = [rdMolDescriptors.GetMorganFingerprintAsBitVect(m_test,radius=2, bitInfo=bi, nBits=256) for m_test in mols_test]
#fingerprints--Morgan
#what is the meaning of 'radius', 'bi'?

In [56]:
#convert fingerprints into binary
nparray_fps_train = []
for fp in fps_train:
    arr = np.zeros((1,),dtype = int)#zeros vector
    DataStructs.ConvertToNumpyArray(fp, arr)
    nparray_fps_train.append(arr)
nparray_fps_test = []
for fp in fps_test:
    arr = np.zeros((1,),dtype = int)#zeros vector
    DataStructs.ConvertToNumpyArray(fp, arr)
    nparray_fps_test.append(arr)

In [57]:
'''MODEL BUILD'''
#Neural Network Architechture
model = Sequential([
    Dense(256, input_shape=(256,), activation='relu'),
    Dense(128, activation='sigmoid'),
    Dense(64, activation='sigmoid'),
    Dense(32, activation='sigmoid'),
    Dense(16, activation='sigmoid'),
    BatchNormalization(axis=1),
    Dense(4, activation='softmax')
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 16)                528       
_________________________________________________________________
batch_normalization_2 (Batch (None, 16)                64        
_________________________________________________________________
dense_12 (Dense)             (None, 4)                

In [58]:
#Compile the model
model.compile(optimizer=Adam(lr=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#Train the model
model.fit(np.array(nparray_fps_train), np.array(array_charges_train), validation_split=0.1, batch_size=10, epochs=100, shuffle=True, verbose=1)

Train on 900 samples, validate on 100 samples
Epoch 1/100
900/900 [==============================] - 0s 542us/step - loss: 1.3863 - accuracy: 0.2433 - val_loss: 1.4290 - val_accuracy: 0.5900
Epoch 2/100
900/900 [==============================] - 0s 526us/step - loss: 1.3812 - accuracy: 0.3089 - val_loss: 1.4003 - val_accuracy: 0.5900
Epoch 3/100
900/900 [==============================] - 0s 494us/step - loss: 1.3754 - accuracy: 0.3756 - val_loss: 1.3865 - val_accuracy: 0.5900
Epoch 4/100
900/900 [==============================] - 0s 484us/step - loss: 1.3705 - accuracy: 0.4244 - val_loss: 1.3787 - val_accuracy: 0.5900
Epoch 5/100
900/900 [==============================] - 0s 515us/step - loss: 1.3650 - accuracy: 0.4800 - val_loss: 1.3719 - val_accuracy: 0.5900
Epoch 6/100
900/900 [==============================] - 0s 514us/step - loss: 1.3577 - accuracy: 0.5456 - val_loss: 1.3642 - val_accuracy: 0.5900
Epoch 7/100
900/900 [==============================] - 1s 571us/step - loss: 1.3516 

Epoch 57/100
900/900 [==============================] - 1s 738us/step - loss: 0.4638 - accuracy: 0.8956 - val_loss: 0.7344 - val_accuracy: 0.7600
Epoch 58/100
900/900 [==============================] - 1s 735us/step - loss: 0.4466 - accuracy: 0.8944 - val_loss: 0.7324 - val_accuracy: 0.7600
Epoch 59/100
900/900 [==============================] - 1s 654us/step - loss: 0.4399 - accuracy: 0.9122 - val_loss: 0.7270 - val_accuracy: 0.7600
Epoch 60/100
900/900 [==============================] - 1s 723us/step - loss: 0.4517 - accuracy: 0.8944 - val_loss: 0.7288 - val_accuracy: 0.7600
Epoch 61/100
900/900 [==============================] - 1s 725us/step - loss: 0.4269 - accuracy: 0.9033 - val_loss: 0.7231 - val_accuracy: 0.7500
Epoch 62/100
900/900 [==============================] - 1s 648us/step - loss: 0.4388 - accuracy: 0.8989 - val_loss: 0.7198 - val_accuracy: 0.7600
Epoch 63/100
900/900 [==============================] - 1s 651us/step - loss: 0.4211 - accuracy: 0.9056 - val_loss: 0.7169 -

In [59]:
#Predict new molecule charges
predictions = model.predict(np.array(nparray_fps_test), batch_size=1, verbose=1)


10/10 [==============================] - 0s 600us/step


In [66]:
#Print the charge predictions
values = []
possib = []
for prediction in predictions:
    prediction = list(prediction)
    values.append(prediction.index(max(prediction)))
    possib.append(prediction[values[-1]])
def switch(value):
    case = {
        0 : '0',
        1 : '1',
        2 : '-1',
        3 : '-2',
        4 : '-3'
    }
    return case.get(value, None)
for i in range(len(values)):
    print('charge:', switch(values[i]),' possibility: ',possib[i])

charge: 0  possibility:  0.79426265
charge: 1  possibility:  0.9309378
charge: -1  possibility:  0.94840187
charge: 1  possibility:  0.9655982
charge: 1  possibility:  0.85117674
charge: 1  possibility:  0.8405196
charge: -1  possibility:  0.8367821
charge: 1  possibility:  0.8486453
charge: 0  possibility:  0.8785551
charge: 1  possibility:  0.9571373


In [ ]:
'''default parameters
charge: 0  possibility:  0.79426265
charge: 1  possibility:  0.9309378
charge: -1  possibility:  0.94840187
charge: 1  possibility:  0.9655982
charge: 1  possibility:  0.85117674
charge: 1  possibility:  0.8405196
charge: -1  possibility:  0.8367821
charge: 1  possibility:  0.8486453
charge: 0  possibility:  0.8785551
charge: 1  possibility:  0.9571373
'''